In [20]:
import torch
from torch.utils.data import Dataset
import torchvision
import torchvision.transforms as transforms
import numpy as np
from  torch.nn.modules.upsampling import Upsample
from matplotlib import pyplot as plt
import torch.nn as nn
import torch.nn.functional as F

In [21]:
seed = 1234
np.random.seed(seed)
torch_gen = torch.Generator()
torch_gen.manual_seed(seed)
torch.manual_seed(seed)

In [22]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

Using device: cuda



Initialize AlexNet

In [23]:
model = torchvision.models.AlexNet(num_classes = 10)
# model.classifier = Sequential( 
    # Example of modifying classifier, add layers here
# )
model = model.to(device)

In [24]:
for param in model.features.parameters():
    param.requires_grad = False

In [25]:
print(f'Trainable Parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad)}')

Trainable Parameters: 54575114


In [26]:
print(f'Total Parameters: {sum(p.numel() for p in model.parameters())}')

Total Parameters: 57044810


In [27]:
m = model
sum(p.numel() for p in m.parameters() if p.requires_grad) / sum(p.numel() for p in m.parameters())

0.9567060351327317

In [28]:
model

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

Download CIFAR-10

In [29]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Resize(size = (224,224)),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

unsplit_trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform = transform)
trainset, valset = torch.utils.data.random_split(unsplit_trainset, [40000, 10000])
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                        download=True, transform = transform)

Files already downloaded and verified
Files already downloaded and verified


In [30]:
len(valset)

10000

In [31]:
# plt.imshow(np.transpose(testset[1][0].numpy(),(1,2,0)))

In [32]:
seed = 1234
np.random.seed(seed)
torch_gen = torch.Generator()
torch_gen.manual_seed(seed)
torch.manual_seed(seed)
noise_level = 1
batch_size = 256 #4


trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2,)


valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

In [33]:
NUM_EPOCHS = 40 # 40

In [38]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
optimizer = optim.Adam(model.parameters(), lr=0.0001)  

In [35]:
import datetime
start_time = str(datetime.datetime.now())

In [36]:
best_vloss = 1_000_000.
timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')

for epoch in range(NUM_EPOCHS):  # loop over the dataset multiple times
    print('EPOCH {}:'.format(epoch + 1))
    running_loss = 0.0
    model.train(True)
    avg_loss = 0.0
    last_loss = 0.0
    for i, data in enumerate(trainloader):
        # print(i)
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            last_loss = running_loss / 2000
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0
    avg_loss = last_loss
    running_vloss = 0.0
    
    model.eval()

    with torch.no_grad():
        for i, vdata in enumerate(valloader):
            vinputs, vlabels = vdata[0].to(device), vdata[1].to(device)
            voutputs = model(vinputs)
            vloss = criterion(voutputs, vlabels)
            running_vloss += vloss

    avg_vloss = running_vloss / (i + 1)
    print('LOSS train {} valid {}'.format(avg_loss, avg_vloss))
    
    if avg_vloss < best_vloss:
        best_vloss = avg_vloss
        model_path = 'model_{}_{}'.format(timestamp, epoch)
        torch.save(model.state_dict(), model_path)
    # torch.save({
    #         'epoch': epoch,
    #         'model_state_dict': model.state_dict(),
    #         'optimizer_state_dict': optimizer.state_dict(),
    #         'loss': loss,
    #         }, './' + start_time + '/' + PATH)
print('Finished Training')

EPOCH 1:
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
LOSS train 0.0 valid 1.8424618244171143
Finished Training


In [37]:
# model = torchvision.models.AlexNet(num_classes = 10)
# model.load_state_dict(torch.load('model_20240524_025323_0'))
# model = model.to(device)

In [39]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        # calculate outputs by running images through the network
        outputs = model(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')
# model_path = 'model_{}_{}_{}'.format(timestamp, "final", 100 * correct // total})
# torch.save(model.state_dict(), model_path)

Accuracy of the network on the 10000 test images: 32 %


In [41]:
model_path = 'model_{}_{}'.format(timestamp, "final")
torch.save(model.state_dict(), model_path)